# Using projectile motion data, let's predict what initial velocity is required when firing at a given angle to reach the intended target distance.

## 1. Import Dependencies and Read Data into Dataframe

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import torch
from torch import nn
from tqdm import tqdm
import os
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score
from sklearn.model_selection import train_test_split

In [3]:
# get current working directory
os.getcwd()

'/Users/jasonsheinkopf/Library/CloudStorage/GoogleDrive-jasonsheinkopf@gmail.com/My Drive/Projects/Python/projectile_motion'

In [4]:
# check if running in google colab
try:
    import google.colab
    from google.colab import drive
    # mount google drive
    drive.mount('/content/drive')
    # Set the working directory
    os.chdir('/content/drive/MyDrive/Projects/Python/projectile_motion')
    # make device agnostic code
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f'Running in Google Colab with device: {device}')
except ModuleNotFoundError:
    device = torch.device("mps")
    print(f'Running Locally with device: {device}')

Running Locally with device: mps


In [5]:
# filepath to pickle data file
# path = 'data/results_small.pkl'
# path = 'data/results_mid.pkl'
path = 'data/results_rand_1000.pkl'
# path = 'data/results_large.pkl'

try:
  with open(path, 'rb') as file:
    # load results list from pickle
    results = pickle.load(file)
    print('Pickle file loaded.')
except FileNotFoundError:
  print('Pickle file not found!')

Pickle file loaded.


In [6]:
# create df from dictionaries in list and view first 5 entries
input_df = pd.DataFrame(results)
input_df.head()

,vel_start,angle,distance
0,0.962451,0.200149,1.924891
1,3.693698,0.204666,7.387349
2,0.306314,0.867718,0.612558
3,0.581955,0.145350,1.163905
4,0.005090,43.872724,0.007338


In [7]:
# save data to csv
input_df.to_csv('input_data.csv')

In [8]:
# sort by distance
input_df = input_df.sort_values(by='distance').reset_index(drop=True)
input_df.head()

,vel_start,angle,distance
0,0.001289,52.352440,0.001575
1,0.005090,43.872724,0.007338
2,0.007521,7.666638,0.014908
3,0.009725,21.237456,0.018129
4,1.134938,89.978315,0.024914


In [9]:
# visualize data
fig = px.scatter_3d(input_df, x='distance', y='angle', z='vel_start',
                    # color=df['distance'].apply(lambda x: 'white' if target - threshold <= x <= target + threshold else 'blue'),
                    labels={'distance': 'Distance', 'angle': 'Angle', 'vel_start': 'Velocity'})

fig.update_traces(marker_size=3)

# Show the interactive plot
fig.show()

## 2. Polynomial Regression

### 2.1 Fit n-degree polynomial regression model to the data and visualize

In [10]:
# extract features and target from the input DataFrame
X = input_df[['angle', 'distance']]
y = input_df['vel_start']

# Split the data into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# create n-polynomial model
n = 3
poly_model = make_pipeline(
    PolynomialFeatures(degree=n),
    LinearRegression()
)

# fit the model
poly_model.fit(X_train, y_train)

# create a meshgrid for the surface plot
distance_surf, angle_surf = np.meshgrid(
    np.linspace(input_df['distance'].min(), input_df['distance'].max(), 100),
    np.linspace(input_df['angle'].min(), input_df['angle'].max(), 100)
)

surface_data = np.column_stack((angle_surf.ravel(), distance_surf.ravel()))

# predict the values for the surface plot
vel_start_surf = poly_model.predict(surface_data)
vel_start_surf = vel_start_surf.reshape(angle_surf.shape)

# Visualize training data
fig = px.scatter_3d(X_train.assign(vel_start=y_train), x='distance', y='angle', z='vel_start',
                    labels={'distance': 'Distance', 'angle': 'Angle', 'vel_start': 'Velocity'})

# Add test data to the scatter plot in red
fig.add_trace(px.scatter_3d(X_test.assign(vel_start=y_test), x='distance', y='angle', z='vel_start',
                             color_discrete_sequence=['red']).data[0].update(marker_size=2))

fig.update_traces(marker_size=3)

# add surface plot for the fitted polynomial model
fig.add_trace(go.Surface(x=distance_surf, y=angle_surf, z=vel_start_surf, opacity=0.7, colorscale='YlOrRd'))

# get the maximum value of velocity from the original data
max_velocity_data = input_df['vel_start'].max()

# set the z-axis range to limit the displayed values
fig.update_layout(scene=dict(zaxis=dict(range=[0, max_velocity_data])))

# show the interactive plot
fig.show()

/Users/jasonsheinkopf/miniforge3/envs/projectile/lib/python3.12/site-packages/sklearn/base.py:465: UserWarning:

X does not have valid feature names, but PolynomialFeatures was fitted with feature names



### 2.2 Predict on test set using polynomial model and visualize results

In [11]:
# predict on the test set
y_pred = poly_model.predict(X_test)
# Evaluate the model performance
mse = mean_squared_error(y_test, y_pred)
# Print evaluation metrics
print(f'Degree-{n} - Polynomial Regression')
print(f'Mean Squared Error on Test Set: {mse}')

Degree-3 - Polynomial Regression
Mean Squared Error on Test Set: 0.1734878108829473


### 2.3 Search for optimal polynomial regression degree

In [12]:
# initial best mse set to high number
best_mse = 100
best_n = None

for degree in range(20):
  # Train a polynomial regression model
  model = make_pipeline(
      PolynomialFeatures(degree=degree),
      LinearRegression()
  )

  model.fit(X_train, y_train)

  # Predict on the test set
  y_pred = model.predict(X_test)

  # Evaluate the model performance
  mse = mean_squared_error(y_test, y_pred)

  # Print evaluation metrics
  print(f'{degree} degree. Mean Squared Error on Test Set: {mse}')

  # update best
  if mse < best_mse:
    best_mse = mse
    best_n = degree

print(f'\nBest performing polynomial regression is {best_n}-degree with {best_mse} mean squared error.')

# create results dictionary to compare models
model_mse = {f'{best_n}-polynomial': best_mse}

0 degree. Mean Squared Error on Test Set: 1.2604176629866601
1 degree. Mean Squared Error on Test Set: 0.595997844058635
2 degree. Mean Squared Error on Test Set: 0.17968777708405362
3 degree. Mean Squared Error on Test Set: 0.1734878108829473
4 degree. Mean Squared Error on Test Set: 0.09989652860192379
5 degree. Mean Squared Error on Test Set: 0.09616598233520492
6 degree. Mean Squared Error on Test Set: 0.07333939791215677
7 degree. Mean Squared Error on Test Set: 0.1604859478801287
8 degree. Mean Squared Error on Test Set: 18.400247009159195
9 degree. Mean Squared Error on Test Set: 0.3211569630827916
10 degree. Mean Squared Error on Test Set: 0.6123828387460175
11 degree. Mean Squared Error on Test Set: 0.5627820441253062
12 degree. Mean Squared Error on Test Set: 0.47596269485148085
13 degree. Mean Squared Error on Test Set: 0.6178117990604879
14 degree. Mean Squared Error on Test Set: 0.5383978089641702
15 degree. Mean Squared Error on Test Set: 8.582440495157245
16 degree. Mean

## 3. Convert Data To Tensors

### 3.1 Normalize DataFrame Values
Neural networks work better with tensors on the same scale [0, 1]

In [13]:
def normalize_df_to_tensors(df):
  '''Normalized values to range [0, 1] and returns as X, y tensors as well as dict of scaling factors.'''
  # create new column for the max value of each variable
  df['max_angle'] = df['angle'].max()
  df['max_distance'] = df['distance'].max()
  df['max_vel_start'] = df['vel_start'].max()

  # divide each column by the largest value in each column, so they ranges [0, 1] and save to new columns
  df['angle_norm'] = df['angle'] / df['max_angle']
  df['distance_norm'] = df['distance'] / df['max_distance']
  df['vel_start_norm'] = df['vel_start'] / df['max_vel_start']

  # check value range again to make sure they range [0, 1]
  print(f"Non-normalized max values: Angle: {df['angle'].max()}, Dist: {df['distance'].max()}, Vel: {df['vel_start'].max()}")
  print(f"Normalized max values: Angle: {df['angle_norm'].max()}, Dist: {df['distance_norm'].max()}, Vel: {df['vel_start_norm'].max()}")

  # extract values from normalized df
  X_values = df[['angle_norm', 'distance_norm']].values
  y_values = df['vel_start_norm'].values

  # convert to tensors
  X = torch.tensor(X_values, dtype=torch.float32)
  y = torch.tensor(y_values, dtype=torch.float32).unsqueeze(dim=1)

  return X, y, df

In [14]:
input_df.head()

,vel_start,angle,distance
0,0.001289,52.352440,0.001575
1,0.005090,43.872724,0.007338
2,0.007521,7.666638,0.014908
3,0.009725,21.237456,0.018129
4,1.134938,89.978315,0.024914


In [15]:
# convert dataframe to tensors
X, y, normalized_df = normalize_df_to_tensors(input_df)

Non-normalized max values: Angle: 89.97831473697605, Dist: 388.7588524011709, Vel: 3.9995145777678025
Normalized max values: Angle: 1.0, Dist: 1.0, Vel: 1.0


In [16]:
input_df.head()

,vel_start,angle,distance,max_angle,max_distance,max_vel_start,angle_norm,distance_norm,vel_start_norm
0,0.001289,52.352440,0.001575,89.978315,388.758852,3.999515,0.581834,0.000004,0.000322
1,0.005090,43.872724,0.007338,89.978315,388.758852,3.999515,0.487592,0.000019,0.001273
2,0.007521,7.666638,0.014908,89.978315,388.758852,3.999515,0.085205,0.000038,0.001881
3,0.009725,21.237456,0.018129,89.978315,388.758852,3.999515,0.236029,0.000047,0.002432
4,1.134938,89.978315,0.024914,89.978315,388.758852,3.999515,1.000000,0.000064,0.283769


In [17]:
# check tensor length
len(X), len(y)

(1000, 1000)

In [18]:
# check tensor size
X.size(), y.size()

(torch.Size([1000, 2]), torch.Size([1000, 1]))

In [19]:
# check first values
X[0], y[0]

(tensor([5.8183e-01, 4.0508e-06]), tensor([0.0003]))

In [20]:
# check first value shapes
X[0].shape, y[0].shape

(torch.Size([2]), torch.Size([1]))

In [21]:
# check first value number of dimensions
print(f'X has {X[0].ndim} dimensions | y has {y[0].ndim} dimensions')

X has 1 dimensions | y has 1 dimensions


In [22]:
# # y shape is a scalar, so we should unsqueeze it so it will match model output
# y = y.unsqueeze(dim=1)
# y[0], y[0].shape

In [23]:
# # check first value number of dimensions
# print(f'X has {X[0].ndim} dimensions | y has {y[0].ndim} dimensions')

In [24]:
# check first 5 tensor values
X[:5], y[:5]

(tensor([[5.8183e-01, 4.0508e-06],
         [4.8759e-01, 1.8876e-05],
         [8.5205e-02, 3.8349e-05],
         [2.3603e-01, 4.6634e-05],
         [1.0000e+00, 6.4086e-05]]),
 tensor([[0.0003],
         [0.0013],
         [0.0019],
         [0.0024],
         [0.2838]]))

In [25]:
def denormalize_tensors_to_df(df, X: torch.Tensor, y: torch.Tensor, pred: torch.Tensor=None) -> pd.DataFrame:
  '''Convert tensor values back to their original scale and save to dataframe. X[angle, distance] y[velocity]'''
  # multiply by scale factor and convert to numpy arrays
  angle_np = (X[:, 0] * df.max_angle[0]).to('cpu').numpy()
  distance_np = (X[:, 1] * df.max_distance[0]).to('cpu').numpy()
  vel_start_np = (y * df.max_vel_start[0]).to('cpu').numpy().squeeze()

  # create df for denormalized values
  denorm_df = pd.DataFrame(data={'vel_start': vel_start_np, 'angle': angle_np, 'distance': distance_np})

  # if prediction is provided
  if pred is not None:
    # denormalize
    prediction_np = (pred * df.max_vel_start[0]).to('cpu').numpy().squeeze()
    # add to the df
    denorm_df['vel_preds'] = prediction_np

  return denorm_df

In [26]:
# denormalize tensors
denorm_df = denormalize_tensors_to_df(input_df, X, y)
denorm_df.head()

,vel_start,angle,distance
0,0.001289,52.352440,0.001575
1,0.005090,43.872726,0.007338
2,0.007521,7.666638,0.014908
3,0.009725,21.237457,0.018129
4,1.134938,89.978317,0.024914


In [27]:
# compare to original df
input_df.head()

,vel_start,angle,distance,max_angle,max_distance,max_vel_start,angle_norm,distance_norm,vel_start_norm
0,0.001289,52.352440,0.001575,89.978315,388.758852,3.999515,0.581834,0.000004,0.000322
1,0.005090,43.872724,0.007338,89.978315,388.758852,3.999515,0.487592,0.000019,0.001273
2,0.007521,7.666638,0.014908,89.978315,388.758852,3.999515,0.085205,0.000038,0.001881
3,0.009725,21.237456,0.018129,89.978315,388.758852,3.999515,0.236029,0.000047,0.002432
4,1.134938,89.978315,0.024914,89.978315,388.758852,3.999515,1.000000,0.000064,0.283769


### 3.2 Create Train / Test Split

In [28]:
# combine X and y into a tuple so they stay grouped together
dataset = torch.utils.data.TensorDataset(X, y)

In [29]:
# check that first value tensor matches df
dataset[0], input_df.head(1)

((tensor([5.8183e-01, 4.0508e-06]), tensor([0.0003])),
    vel_start     angle  distance  max_angle  max_distance  max_vel_start  \
 0   0.001289  52.35244  0.001575  89.978315    388.758852       3.999515   
 
    angle_norm  distance_norm  vel_start_norm  
 0    0.581834       0.000004        0.000322  )

In [30]:
# set size of train and test stes
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

In [31]:
# create train and test datasets
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
len(train_dataset), len(test_dataset)

(800, 200)

## 4. Build a Neural Network

In [32]:
# try a simple model
class RegressionModelV0(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    # initialize parent class nn.Module
    super().__init__()
    # create linear layer to feed the two X values into (angle, distance)
    self.layer1 = nn.Linear(input_size, hidden_size)
    # create nonlinear activation function
    self.relu = nn.ReLU()
    # create second linear layer to feed the hidden layers into the output
    self.layer2 = nn.Linear(hidden_size, output_size)

  def forward(self, x):
    x = self.layer1(x)
    x = self.relu(x)
    x = self.layer2(x)
    return x

In [33]:
# set manual seed for reproducibility
torch.manual_seed(57)
# create instance of model 0
model_0 = RegressionModelV0(input_size=2, hidden_size=20, output_size=1).to(device)

In [34]:
# define a loss function (mean squared error)
loss_fn = nn.MSELoss()
# choose optimiser
optimizer = torch.optim.Adam(model_0.parameters(), lr=0.001)

## 5. Make Predictions with Untrained Model

### 5.1 Try feeding first data point through model to get prediction

In [35]:
# get a sample for the testing data
X_sample, y_sample = dataset[0]
# put sample on device
X_sample, y_sample = X_sample.to(device), y_sample.to(device)
X_sample, y_sample

(tensor([5.8183e-01, 4.0508e-06], device='mps:0'),
 tensor([0.0003], device='mps:0'))

In [36]:
# set model to evaluation (no learning) mode
model_0.eval()

RegressionModelV0(
  (layer1): Linear(in_features=2, out_features=20, bias=True)
  (relu): ReLU()
  (layer2): Linear(in_features=20, out_features=1, bias=True)
)

In [37]:
# turn off gradient tracking (which is only used for learning)
with torch.no_grad():
  # make a prediction
  predicted_y = model_0(X_sample)

In [38]:
# see the predicted value and true vales
print(f'The untrained model predicted normalzed {y_sample.item()} and the true value was {predicted_y.item()}')

The untrained model predicted normalzed 0.0003223177045583725 and the true value was -0.45733892917633057


This makes sense since the model has not been trained.

In [39]:
# check shape of true y and predicted y
y_sample.shape, predicted_y.shape

(torch.Size([1]), torch.Size([1]))

In [40]:
y_sample.ndim, predicted_y.ndim

(1, 1)

In [41]:
# calculate test loss
loss_sample = loss_fn(predicted_y, y_sample)
loss_sample

tensor(0.2095, device='mps:0')

In [42]:
# check the sample X and its shape
X_sample, X_sample.shape

(tensor([5.8183e-01, 4.0508e-06], device='mps:0'), torch.Size([2]))

In [43]:
# add dimension to individual X sample to match tensor of a batch of many sample values which we will usually use
X_sample = X_sample.unsqueeze(dim=0)
X_sample.shape

torch.Size([1, 2])

In [44]:
# denormalize sample to df and check normalization -> denormalization results in original value
denorm_sample_df = denormalize_tensors_to_df(input_df, X_sample, y_sample, predicted_y)
print('Input df')
print(input_df.head(1))
print('\nModel Prediction')
print(denorm_sample_df.head(1))

Input df
   vel_start     angle  distance  max_angle  max_distance  max_vel_start  \
0   0.001289  52.35244  0.001575  89.978315    388.758852       3.999515   

   angle_norm  distance_norm  vel_start_norm  
0    0.581834       0.000004        0.000322  

Model Prediction
   vel_start     angle  distance  vel_preds
0   0.001289  52.35244  0.001575  -1.829134


### 5.2 Predict Entire Test Dataset

In [45]:
def predict_all_test(test_dataset, model, input_df, loss_fn):
    # create list to store results dfs
    results_df_list = []
    # create variable to track loss
    test_loss = 0
    # iterate over all samples in test dataset
    for sample in tqdm(test_dataset):
      # extract X and y
      X_sample, y_sample = sample
      # put sample on target device
      X_sample, y_sample = X_sample.to(device), y_sample.to(device)
      # set model to eval mode
      model.eval()
      # turn off gradient tracking
      with torch.no_grad():
        # predict
        predicted_y = model(X_sample)
        # calculate loss and add to running todal
        test_loss += loss_fn(predicted_y, y_sample)
        # add dimension to X
        X_sample = X_sample.unsqueeze(dim=0)
        # denormalize result and save to df
        denorm_sample_df = denormalize_tensors_to_df(input_df, X_sample, y_sample, predicted_y)
        # append individual sample df to list
        results_df_list.append(denorm_sample_df)
    # concatenate list of results dataframes
    test_pred_df = pd.concat(results_df_list, axis=0, ignore_index=True)
    # divide accumulated loss by num of items
    test_loss /= len(test_dataset)

    return test_pred_df, test_loss.item()

In [46]:
# predict entire test data with current model
test_preds_df, test_loss = predict_all_test(test_dataset, model_0, input_df, loss_fn)
test_preds_df.head()

  0%|          | 0/200 [00:00<?, ?it/s]

100%|██████████| 200/200 [00:00<00:00, 596.44it/s]


,vel_start,angle,distance,vel_preds
0,2.411193,11.925761,61.337898,-1.398370
1,3.253272,83.420883,60.757290,-1.871822
2,2.777755,37.465614,189.608948,-1.093296
3,2.166662,64.680687,91.733414,-1.609364
4,3.322473,35.892471,266.468170,-0.821570


In [47]:
def plot_results(input_df, train_dataset, test_preds_df):
    # Denormalize datasets to DataFrame
    train_df = denormalize_tensors_to_df(input_df, train_dataset[:][0], train_dataset[:][1])

    marker_size = 3

    # Create scatter plot for train data in blue
    fig = px.scatter_3d(train_df, x='distance', y='angle', z='vel_start',
                              labels={'distance': 'Distance', 'angle': 'Angle', 'vel_start': 'Velocity'},
                              title='Train Data', size_max=marker_size)

    # Update marker size for test data
    fig.update_traces(marker=dict(size=marker_size))

    # Add test data to the scatter plot in red
    fig.add_trace(px.scatter_3d(test_preds_df, x='distance', y='angle', z='vel_preds',
                             color_discrete_sequence=['red']).data[0].update(marker_size=2))

    # Update marker size for predictions
    fig.update_traces(marker=dict(size=marker_size))

    # Show the interactive plot
    fig.show()

In [48]:
# plot results of untrained nn
plot_results(input_df, train_dataset, test_preds_df)

In [49]:
print(f'Model 0 with no training test loss: {test_loss}')

Model 0 with no training test loss: 0.8636081218719482


In [50]:
# check min and max velocity values
test_preds_df.vel_start.min(), test_preds_df.vel_start.max()

(0.009725187, 3.9797838)

## 6. Training Neural Network

In [51]:
train_dataset[0]

(tensor([0.5217, 0.1252]), tensor([0.3429]))

In [52]:
# check model device
next(model_0.parameters()).device

device(type='mps', index=0)

In [53]:
# count epochs across multiple runs of training cell
total_training_epochs = 0

In [54]:
%%time
torch.manual_seed(42)

epochs = 10000

epoch_count = []
loss_values = []
test_loss_values = []

# extract train and test sets
X_train, y_train = train_dataset[:]
X_test, y_test = test_dataset[:]

# put train and test sets on device
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

# iterate over range of epochs
for epoch in range(epochs):
  total_training_epochs += 1
  ### Train
  model_0.train()
  # forward pass
  y_pred = model_0(X_train)
  # calculate loss
  loss = loss_fn(y_pred, y_train)
  # optimizer zero grade
  optimizer.zero_grad()
  # backpropagation
  loss.backward()
  # step
  optimizer.step()

  ### Test
  model_0.eval()
  # turn off gradient tracking
  with torch.inference_mode():
    # forward pass
    test_pred = model_0(X_test)
    # calculate loss
    test_loss = loss_fn(test_pred, y_test)

  if epoch % (epochs / 10) == 0:
    epoch_count.append(epoch)
    loss_values.append(loss)
    test_loss_values.append(test_loss)
    print(f'Epoch: {epoch} | Loss: {loss} | Test Loss: {test_loss}')

print(f'{total_training_epochs=}')

Epoch: 0 | Loss: 0.8664306402206421 | Test Loss: 0.8450151085853577
Epoch: 1000 | Loss: 0.021547526121139526 | Test Loss: 0.021936992183327675
Epoch: 2000 | Loss: 0.014035899192094803 | Test Loss: 0.013965514488518238
Epoch: 3000 | Loss: 0.011873522773385048 | Test Loss: 0.01096267532557249
Epoch: 4000 | Loss: 0.010553278960287571 | Test Loss: 0.00895525049418211
Epoch: 5000 | Loss: 0.010124830529093742 | Test Loss: 0.008175985887646675
Epoch: 6000 | Loss: 0.009812450036406517 | Test Loss: 0.007890215143561363
Epoch: 7000 | Loss: 0.009598728269338608 | Test Loss: 0.007698007859289646
Epoch: 8000 | Loss: 0.009458285756409168 | Test Loss: 0.007617454510182142
Epoch: 9000 | Loss: 0.009181791916489601 | Test Loss: 0.007464532740414143
total_training_epochs=10000
CPU times: user 16.3 s, sys: 892 ms, total: 17.2 s
Wall time: 16.5 s


In [55]:
# predict entire test data with current model
test_preds_df, test_loss = predict_all_test(test_dataset, model_0, input_df, loss_fn)

# save results to dict
model_mse['neural network'] = test_loss

# plot test data
plot_results(input_df, train_dataset, test_preds_df)

100%|██████████| 200/200 [00:00<00:00, 536.49it/s]


In [56]:
# compare best polynomial model to neural network
for key, value in model_mse.items():
  print(f'{key} model mse {value}')

6-polynomial model mse 0.07333939791215677
neural network model mse 0.00721050426363945


## 7. Using the NN to hit a target at given distance and angle

In [62]:
def nn_predict (model_0, angle, distance, input_df):
  # normalize features
  normalized_angle = angle / input_df['max_angle'][0]
  normalized_distance = distance / input_df['max_distance'][0]

  # create feature tensor
  X_trial = torch.tensor([[normalized_angle, normalized_distance]], dtype=torch.float32).to(device)

  model_0.eval()
  # turn off gradient tracking
  with torch.inference_mode():
    # forward pass
    y_trial_normalized = model_0(X_trial).item()

  y_trial = y_trial_normalized * input_df['max_vel_start'][0]

  return y_trial

In [63]:
# 0 < angle < 90
angle = 80

# 0 < distance < 400
distance = 400

# predict with nn
y_trial = nn_predict(model_0, angle, distance, input_df)

y_trial

5.0167627685734235

## 8. Save model to pickle to test in environment


In [64]:
torch.save(model, 'model/model_0.pth')
with open('model/input_df.pkl', 'wb') as f:
    pickle.dump(input_df, f)